In [1]:
#8-1 
import numpy as np

def reweight_distribution(original_distribution, tempreature=0.5):
    distribution = np.log(original_distribution)/temperature
    distribution = np.exp(distribution)
    return distribution /np.sum(distribution)

In [2]:
import keras 

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text=open(path).read().lower()
print('size of text:', len(text))

Using TensorFlow backend.
/home/server1/anaconda3/envs/ksa2020/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/server1/anaconda3/envs/ksa2020/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/server1/anaconda3/envs/ksa2020/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/server1/

size of text: 600893


In [3]:
maxlen = 60 #extract sequence to consist of 60 alphabet form test
step =3

sentences = []
next_chars=[]

for i in range(0,len(text)-maxlen, step):
    sentences.append(text[i:i+maxlen])
    next_chars.append(text[i+maxlen])
    #in this point, Because I wrote wrong spell text[i:maxlen], it makes key error at line 28 

print('number of sequences:',len(sentences))

chars=sorted(list(set(text))) #sort word that have independent meaning  and put in list
print('Unique word:',len(chars))
char_indices=dict((char,chars.index(char)) for char in chars)
print(char_indices)
#make a dictionary which mapping word in list with their index 

print('vectorization...')
x=np.zeros((len(sentences),maxlen,len(chars)), dtype=np.bool)
y=np.zeros((len(sentences),len(chars)),dtype=np.bool)
for i,sentence in enumerate(sentences): #enumerate => check the index in Repetitive
    #print(sentence)
    for t,char in enumerate(sentence):
        #print(t,char)
        #print(char_indices[char])
        x[i,t,char_indices[char]]=1
    y[i,char_indices[next_chars[i]]]=1
#one-hot encodeing for sentences

number of sequences: 200278
Unique word: 57
{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '=': 22, '?': 23, '[': 24, ']': 25, '_': 26, 'a': 27, 'b': 28, 'c': 29, 'd': 30, 'e': 31, 'f': 32, 'g': 33, 'h': 34, 'i': 35, 'j': 36, 'k': 37, 'l': 38, 'm': 39, 'n': 40, 'o': 41, 'p': 42, 'q': 43, 'r': 44, 's': 45, 't': 46, 'u': 47, 'v': 48, 'w': 49, 'x': 50, 'y': 51, 'z': 52, 'ä': 53, 'æ': 54, 'é': 55, 'ë': 56}
vectorization...


In [4]:
from keras import layers

model=keras.models.Sequential()
model.add(layers.LSTM(128,input_shape=(maxlen,len(chars))))
model.add(layers.Dense(len(chars),activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [5]:
optimizer=keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy',optimizer=optimizer)

In [6]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas= np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [7]:
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text)-maxlen-1)

for epoch in range(1,60) :
    print('epoch:', epoch)
    history=model.fit(x,y,batch_size=128,epochs=1)
    
    seed_text = text[start_index: start_index+maxlen]
    print('----seed text:"'+seed_text+'"')
    
    for temperature in [0.2,0.5,1.0,1.2]:
        print('----temperature:',temperature)
        generated_text=seed_text
        sys.stdout.write(generated_text)
        
        for i in range(400):
            sampled = np.zeros((1,maxlen,len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0,t,char_indices[char]]=1.
            
            preds=model.predict(sampled, verbose=0)[0]
            next_index=sample(preds, temperature)
            next_char=chars[next_index]
            
            generated_text+=next_char
            generated_text=generated_text[1:]
            
            sys.stdout.write(next_char)
            sys.stdout.flush()
            
        print()

epoch: 1
Instructions for updating:
Use tf.cast instead.
Epoch 1/1
200278/200278 [==============================] - 54s 269us/step - loss: 1.9876
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through for the proper the spirit of the proper the persent and the religions of the propers of the spirit of the spirit and still the proper the spirit of the persent of the proper to the religious one and the spirit and the spirit of the spirit of the spirit of the restinces of the persent and some the precerale the spirit of the spirit of the proper the spirit of the portain the persent the spirit of the
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through for the latter to be exper-and indection of the persent of the relation of the condinction and the persent of the sperity of the way reality of the dett of have the detion of not the perhaps the regard of the senful 

/home/server1/anaconda3/envs/ksa2020/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


rd that the spirit with the schopenhaueres and platories of the law, and not be also that the called by the dispasic of the wh
----temperature: 1.0
the slowly ascending ranks and classes, in which,
through forbingful, in the stip of the self-clurs of levenisher beensely or in all characterisly call works, or loboral a
pourtic of able that it is
is dif origin of its disprise. upon reasions of "profound. i deerness of e. he to unigher
kings and preander, that this would a presuppes doon and arwives with the
friendswich
to those we causation bimilant parampously, is
heart of the morals of vallenn depress
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through formares.


1yiget, finds
concerns us that
he perhals?--and free oneawad, amid the loanest coasi form! how deep our profoundly _kinds, to make dispositic manventh lookar
greatest usmible of the lehadists of himself and means and puelisted the begromes have that volyere merefainure,
morality--and follow to its p

through formerly and all the man and the promise the acts of the senses of the spirit of the senses of the senses of the man the more conception of the sense of the senses of the secret of his problems of the sense of the senses of the secret of the profound the deceived the spirit of the senses of the senses of the socrate of the senses of the belief to the senses of the senses of the proper to the sense o
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through formerly, here all the calle now formerly and his fictions of his responsed to many man of the deceived of the willing to the art of the prevail of the freelof, by desire commens than and its attracitical to the desire to the very deality of the entire under the individuals and feelons of the fark and "the persupers of the acts in the emotion and accordated the obligition of the strong who deceived 
----temperature: 1.0
the slowly ascending ranks and classes, in which,
through fone the
sacrifice of the wo

so willoal werebiec. in frnandans sausify the
slaves of
of be st(parances. kindness, eve
consivelery, soul who lee
not manifest our mal," and one have to garmed to chareular, in general occlised manifest seld giusness, with psome existence world o
epoch: 23
Epoch 1/1
200278/200278 [==============================] - 52s 262us/step - loss: 1.3303
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through former and the consequently to the man in the sense of the superian and conflitually the present in the more the more the far and conceited and such a man of the world and in the look and the suffering to the contempt the sense of the subtle of the sense of the sense of the same one the strength of the present strong the sense of the contemptity of the man in the same the same conscience of the stre
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through former, the man

through foler,
for all
them bleet precably than to nature the changed: actsfores: for "ipus: 
eve turh gebliguous et result
are reflece fully distalcisting of deepul tranpegous virtue-and evious sin is pleasings, when a char stedied theor and superics: there is grateful have no man conscience is
at her insequence moved veness
of condudanle such a spiritual capace of essemomiful
german partialss--it is not
n
epoch: 27
Epoch 1/1
200278/200278 [==============================] - 52s 262us/step - loss: 1.3220
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through former and protection of the strength and according to the secrection of the strength and stronger and superity of the self self-contemplation of the strength and in the contemplation of the struggle and such a strong the last the strength in the strength and anti-father and command and the strength and abvoration of the consequentl

through follow god in mysituse" of the precessone esenves of science, and but for interesting counter, there are--or then arrest
an ignorickness, and possement explanation of
the greatness and literatury and make pusince. this he can thus rovest in dratersiflive quarsxquine, and to depths. the unsuphibre? is love we hen
in the
it, that of it is concepting manoken unsubr"e things elovo. every goining and
mor
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through founderd labe. is idrute of the middle pry hoprsous
not and
forsequences of his promises hichetod; he hasf.
)jumqualting the nigforier-, ruling inbove tamply suffer in "philosophy perhaps, as seepjoy: as
we not to nature to the ohy in fact, it will is, evil? every
dew is objective. "our quitle of cloness: as what of amaut be traigh of
myisely else.


49

=usefm, who call hus ipon
asseai the
expityru
epoch: 35
Epoch 1/1
200278/200278 [==============================] - 53s 262us/step - loss: 1.3115
----seed

through for the same the same the strength as the great the soul, and something as the strength and serve the world of the man and self-consideration of the present world of the strength and the ever the same the same the strength in the same to the same than one will the strength as the same things understand the strength as the sense of the stringed the more inventions of the promise which is not a man an
----temperature: 0.5
the slowly ascending ranks and classes, in which,
through formations of the spiritual consequently and attenuated in the good and actions of "great press of the onesell, we now mankind with his way, as the imagines that all the conditions of the expedient mankind is the stringer a man and brought to sour has a man are stuths of the actions of the self-deligne(ed by the proper of such the scientific festerness of made the more being of the past in the sense
----temperature: 1.0
the slowly ascending ranks and classes, in which,
through for men expecial, as the fut

belond--, to abso ancialtourishms, desposituonal whate are schopenhaueated of naids, sopetifer, does sense.

132
.

12. a licked, "man"--now,
vigile, cease-beage into mean
fronignrious other wornd, that, which ougciditiness, the schoolc:--a dispaint of
this disfine batt not neces of
epoch: 50
Epoch 1/1
200278/200278 [==============================] - 52s 262us/step - loss: 1.2988
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temperature: 0.2
the slowly ascending ranks and classes, in which,
through forms of the most experience of the contempt that the problem of the present in the same superiority of the present and act of the world and more the significaly and senses the presence of the most time to the senses of the spirit in the presence of the presence of man and desire to the subjection of the sensess of the belief in the same time the powerful and constantly and the lives of the contempt
----temperature: 0.5
the slowly ascending ranks and classe

the methorking concernaly he adgre-betverds, exist--this can cordurestening require with
schemirved misseves before do ansay, not i hate contemplary might pleasure--as i can consists of the germans finrism at were he must believe in a one
by the ranes-es.


91

=natured with the
instincts, and "illogics,
frrea wish
for only beaun
----temperature: 1.2
the slowly ascending ranks and classes, in which,
through forgre themgering,
even to be also a onesty very goolate. on
the ey againsing accoro and notion, is at last higher,
pard rake; whenever go orging "neat, foundaus, whon roves of another could gotars! whhere affortanct because let essy finepich--inas thought's "france "values: yeem.


tess moreitioy,
fuly only ere tebastery, arterate tenestates--no esycry, would not dan
whiles of every anti-vanite and
epoch: 58
Epoch 1/1
200278/200278 [==============================] - 53s 262us/step - loss: 1.3007
----seed text:"the slowly ascending ranks and classes, in which,
through fo"
----temper

In [21]:
import matplotlib.pyplot as plt

loss=history.history['loss']
print(type(loss))
print(loss[0])
print(loss[1])
plt.plot(epoch, loss, 'b', label='loss')
plt.title('loss per epochs')
plt.legend()
plt.show()

<class 'list'>
1.301025758829912


IndexError: list index out of range